# Testing Multiple Linear Regression Model on Hold-out Data

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=eP1sWyYgRAuPJmNVYGq8ubmf6SrWQnxdbTMgD2MuVFE&tc=KD-EPbWLf9MmCTga1XnfowniPqs8--Z5VdIIy7reWJc&cc=zL38ayMVqjbYE-cnXPOyiab6m7Y_WpLJ6ryhSMQlbSE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXl7zMSu5nBIc9TpkDElOINDVYfdZdgPfC2Kw8MzmFF4jo0VPBjvPsM

Successfully saved authorization token.


### Import held out data

In [ ]:
# bring in the last 10 percent of data for generalizing
testing = ee.FeatureCollection('users/aspenjkmorgan/k_folds/test')
testing.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-114.189259, 48.363709]},
 'id': '00000000000000000000',
 'properties': {'DPT': 5.050012111663818,
  'PBLH': 903.87353515625,
  'PRES': 91148.5,
  'RH': 59.955150648997744,
  'TMP': 12.624994277954102,
  'WDIR': 201.3000030517578,
  'WIND': 1.7549999952316284,
  'aod': 0.217,
  'pm25': 3.103142857,
  'random': 0.9806150715932024,
  'station': 'Flathead Valley',
  'system:time_start': 1339092000000}}

### Use best split of data for training the model

In [ ]:
# best fold of training data
training = ee.FeatureCollection('users/aspenjkmorgan/k_folds/training_fold_9')
training.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-114.189259, 48.363709]},
 'id': '00000000000000000000',
 'properties': {'DPT': -4.599997043609619,
  'PBLH': 66.51596069335938,
  'PRES': 91613.5,
  'RH': 67.44328718390516,
  'TMP': 0.7449890375137329,
  'WDIR': 234.10000610351562,
  'WIND': 0.5600000023841858,
  'aod': 0.008,
  'pm25': 5.5785,
  'random': 0.006469227008691791,
  'station': 'Flathead Valley',
  'system:time_start': 1325534400000}}

### Helper functions

In [ ]:
# Compute the difference between computed pm25 and actual
def difference(feature):
  diff = ee.Number(feature.get('pm_cal')) \
    .subtract(ee.Number(feature.get('pm25')))
  # Return the feature with the squared difference set to the 'diff' property.
  return feature.set('sq_diff', diff.pow(2))

def getTss(feature):
  buf = ee.Number(feature.get('pm25')).subtract(ee.Number(mean_actual))
  return feature.set('tss', buf.pow(2))

### Multiple Linear Regression Model

In [ ]:
# add constant column to training set for bis
def addConstant(feature):
  return feature.set('constant', 1)
training = training.map(addConstant)

mlr_model = training.reduceColumns(**{
    'reducer': ee.Reducer.linearRegression(**{'numX': 8,'numY': 1}),
    'selectors': ['aod', 'WIND', 'RH', 'PRES', 'WDIR', 'DPT', 'PBLH', 'constant', 'pm25']})

coeffs = ee.Array(mlr_model.get('coefficients')).toList()

def applyLinearRegModel(feature):
  aod_effect = ee.Number(feature.get('aod')).multiply(ee.List(coeffs.get(0)).get(0))
  wind_effect = ee.Number(feature.get('WIND')).multiply(ee.List(coeffs.get(1)).get(0))
  rh_effect = ee.Number(feature.get('RH')).multiply(ee.List(coeffs.get(2)).get(0))
  pres_effect = ee.Number(feature.get('PRES')).multiply(ee.List(coeffs.get(3)).get(0))
  wdir_effect = ee.Number(feature.get('WDIR')).multiply(ee.List(coeffs.get(4)).get(0))
  dpt_effect = ee.Number(feature.get('DPT')).multiply(ee.List(coeffs.get(5)).get(0))
  pblh_effect = ee.Number(feature.get('PBLH')).multiply(ee.List(coeffs.get(6)).get(0))
  y_int = ee.Number(ee.List(coeffs.get(7)).get(0))
  pm_cal = aod_effect.add(wind_effect).add(rh_effect).add(pres_effect).add(wdir_effect).add(dpt_effect).add(pblh_effect).add(y_int)
  return feature.set('pm_cal', pm_cal)

In [ ]:
# get predictions
testing_applied = testing.map(applyLinearRegModel)

In [ ]:
testing_applied.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-114.189259, 48.363709]},
 'id': '00000000000000000000',
 'properties': {'DPT': 5.050012111663818,
  'PBLH': 903.87353515625,
  'PRES': 91148.5,
  'RH': 59.955150648997744,
  'TMP': 12.624994277954102,
  'WDIR': 201.3000030517578,
  'WIND': 1.7549999952316284,
  'aod': 0.217,
  'pm25': 3.103142857,
  'pm_cal': 9.181665691049579,
  'random': 0.9806150715932024,
  'station': 'Flathead Valley',
  'system:time_start': 1339092000000}}

### Calculate Metrics (RMSE and $R^2$)

In [ ]:
mean_actual = testing_applied.aggregate_mean('pm25')
testing_applied = testing_applied.map(difference)
testing_applied = testing_applied.map(getTss)

# RMSE for validation data
testing_rmse = ee.Number(testing_applied.reduceColumns(ee.Reducer.mean(), ['sq_diff']).get('mean')).sqrt()

In [ ]:
print('RMSE: ' + str(testing_rmse.getInfo()))

RMSE: 11.431809878182252


In [ ]:
# get R2
rss = ee.Number(testing_applied.reduceColumns(ee.Reducer.sum(), ['sq_diff']).get('sum'))
tss = ee.Number(testing_applied.reduceColumns(ee.Reducer.sum(), ['tss']).get('sum'))
r2 = ee.Number(1).subtract(rss.divide(tss))

In [ ]:
print('R2: ' + str(r2.getInfo()))

R2: 0.44725984407759656
